In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import chardet

In [3]:
import json

In [4]:
import os

In [5]:
with open("NYT Articles.csv", 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']

In [6]:
data_df = pd.read_csv("NYT Articles.csv", delimiter=',', encoding=encoding)

In [7]:
data_df

,Date,Publication,Headline,Author 1,Authour 2,Political Leaning,News/Opinion,URL
0,03/28/2023,New York Times,M.T.A. Plans to Use Congestion Pricing Funds t...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2023/03/28/nyregion/mt...
1,12/19/2022,New York Times,Can a $10 Billion Highway Fix One of New Jerse...,Tracey Tully,Patrick McGeehan,Left,News,https://www.nytimes.com/2022/12/19/nyregion/ho...
2,10/11/23,New York Times,Yellow Cabs Are Struggling. Congestion Pricing...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/10/11/nyregion/ny...
3,10/06/22,New York Times,New Jersey Governor Seeks to Slow New York’s C...,"Tracey Tully,",Ana Ley,Left,News,https://www.nytimes.com/2022/10/06/nyregion/nj...
4,09/12/22,New York Times,A Plan to Push Cars Out of Manhattan Could Mak...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/09/12/nyregion/ny...
5,08/26/2022,New York Times,"At M.T.A.’s First Congestion Pricing Hearing, ...",Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/08/26/nyregion/co...
6,08/25/2022,New York Times,The Pros and Cons of Congestion Pricing in Man...,James Barron,NaN,Left,News,https://www.nytimes.com/2022/08/25/nyregion/co...
7,08/18/2022,New York Times,Why Drivers Could Soon Pay $23 to Reach Manhattan,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/08/18/nyregion/ny...
8,08/17/2022,New York Times,The M.T.A.’s Money Woes,James Barron,NaN,Left,News,https://www.nytimes.com/2022/08/17/nyregion/th...
9,06/12/22,New York Times,There Has to Be a Better Way to Run the Govern...,Ezra Klein,NaN,Left,Opinion,https://www.nytimes.com/2022/06/12/opinion/tra...


In [8]:
api_key = 'GqErWPG8M2JhGA7CGoGybdb6HdhyufNU'

In [9]:
def get_article_text(url):
    response = requests.get(f'https://api.nytimes.com/svc/article/v1/{url}.json?api-key={api_key}')
    if response.status_code == 200:
        data = response.json()
        article_text = data['response']['docs'][0]['body']
        return article_text
    else:
        return None

In [10]:
data_df['article_text'] = ''

In [11]:
id=0
for i, row in data_df.iterrows():
    url = row['URL']
    article_text = ""

    response = requests.get(f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={url}&api-key={api_key}&fl=web_url,headline,snippet,lead_paragraph")

    if response.status_code == 200:
        data = response.json()
        if data['response']['docs']:
            web_url = data['response']['docs'][0]['web_url']
            print (web_url)
            #Wanted to check that the correct URLs were being taken 
            headline = data['response']['docs'][0]['headline']['main']
            snippet = data['response']['docs'][0]['snippet']
            lead_paragraph = data['response']['docs'][0]['lead_paragraph']
            article_text = f"{headline}\n\n{snippet}\n\n{lead_paragraph}\n"
            # Get article content using web scraping
            article_soup = get_article_text(web_url)
            if article_soup:
                article_content = article_soup.find_all('section', {'class': 'meteredContent css-1r7ky0e'})
                if article_content:
                    for content in article_content:
                        article_text += content.get_text()
    
    print(f"Article {id}: {headline}")
    id+=1
    data_df.at[i, 'article_text'] = article_text + '\n'

https://www.nytimes.com/2023/03/28/nyregion/mta-congestion-pricing-pollution-bronx.html
Article 0: M.T.A. Plans to Use Congestion Pricing Funds to Address Bronx Pollution
https://www.nytimes.com/2022/12/19/nyregion/holland-tunnel-turnpike-extension.html
Article 1: Can a $10 Billion Highway Fix One of New Jersey’s Worst Traffic Jams?
https://www.nytimes.com/2022/10/11/nyregion/nyc-traffic-yellow-cab-tolls.html
Article 2: Yellow Cabs Are Struggling. Congestion Pricing Could Deal a New Blow.
https://www.nytimes.com/2022/10/06/nyregion/nj-biden-congestion-price-toll.html
Article 3: New Jersey Governor Seeks to Slow New York’s Congestion Pricing Effort
https://www.nytimes.com/2022/09/12/nyregion/nyc-congestion-pricing-manhattan-bronx.html
Article 4: A Plan to Push Cars Out of Manhattan Could Make the Bronx’s Air Dirtier
Article 5: A Plan to Push Cars Out of Manhattan Could Make the Bronx’s Air Dirtier
Article 6: A Plan to Push Cars Out of Manhattan Could Make the Bronx’s Air Dirtier
Article

In [12]:
data_df

,Date,Publication,Headline,Author 1,Authour 2,Political Leaning,News/Opinion,URL,article_text
0,03/28/2023,New York Times,M.T.A. Plans to Use Congestion Pricing Funds t...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2023/03/28/nyregion/mt...,M.T.A. Plans to Use Congestion Pricing Funds t...
1,12/19/2022,New York Times,Can a $10 Billion Highway Fix One of New Jerse...,Tracey Tully,Patrick McGeehan,Left,News,https://www.nytimes.com/2022/12/19/nyregion/ho...,Can a $10 Billion Highway Fix One of New Jerse...
2,10/11/23,New York Times,Yellow Cabs Are Struggling. Congestion Pricing...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/10/11/nyregion/ny...,Yellow Cabs Are Struggling. Congestion Pricing...
3,10/06/22,New York Times,New Jersey Governor Seeks to Slow New York’s C...,"Tracey Tully,",Ana Ley,Left,News,https://www.nytimes.com/2022/10/06/nyregion/nj...,New Jersey Governor Seeks to Slow New York’s C...
4,09/12/22,New York Times,A Plan to Push Cars Out of Manhattan Could Mak...,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/09/12/nyregion/ny...,A Plan to Push Cars Out of Manhattan Could Mak...
5,08/26/2022,New York Times,"At M.T.A.’s First Congestion Pricing Hearing, ...",Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/08/26/nyregion/co...,\n
6,08/25/2022,New York Times,The Pros and Cons of Congestion Pricing in Man...,James Barron,NaN,Left,News,https://www.nytimes.com/2022/08/25/nyregion/co...,\n
7,08/18/2022,New York Times,Why Drivers Could Soon Pay $23 to Reach Manhattan,Ana Ley,NaN,Left,News,https://www.nytimes.com/2022/08/18/nyregion/ny...,\n
8,08/17/2022,New York Times,The M.T.A.’s Money Woes,James Barron,NaN,Left,News,https://www.nytimes.com/2022/08/17/nyregion/th...,\n
9,06/12/22,New York Times,There Has to Be a Better Way to Run the Govern...,Ezra Klein,NaN,Left,Opinion,https://www.nytimes.com/2022/06/12/opinion/tra...,\n


In [13]:
print(data_df.at[2, 'article_text'])

Yellow Cabs Are Struggling. Congestion Pricing Could Deal a New Blow.

The tolling program could push up fares and shrink taxi demand, the M.T.A. says. Cabdrivers are also seeking their own fare hike of up to 23 percent.

New York City’s taxi industry has faced a drumbeat of challenges over the past decade, from predatory loans and the rise of ride-share apps to a global pandemic that dried up business. Now, an ambitious government effort to boost public transit and push commuters out of cars could be the latest existential threat confronting the iconic yellow cab.


